# SLAM tutorial

SLAM and factor graphs tutorial prepared for the ORIentate Seminars

Author: Matias Mattamala (matias@robots.ox.ac.uk, [mmattamala@github](https://github.com/mmattamala))

Date: 23/11/2023


> ⚠️ **Warning**: Before proceeding, make sure you installed all the required dependencies in the [`requirements.txt`](../requirements.txt) file!


## Preliminaries

In [ ]:
%pip install -e ../

### Part 1: Inspecting a perfect SLAM graph

In [ ]:
import slam_tutorial
from slam_tutorial.io import load_pose_graph
graph = load_pose_graph(slam_tutorial.ASSETS_DIR + "/ground_truth.slam")

In [ ]:
import slam_tutorial.visualization as vis
vis.show_pose_graph(graph)

### Part 2: A more realistic pose graph

In [ ]:
import slam_tutorial.pose_graph as pg
graph_with_drift = pg.add_odometry_drift(graph, noise_per_m=0.1, axis="xy", drift_type="random_walk")
vis.show_pose_graph(graph_with_drift)

### Part 3: Solving a pose graph with factor graphs

In [ ]:
pose_graph = pg.create_test_pose_graph(noise_per_m=0.1, axis="xy", drift_type="random_walk")
vis.show_pose_graph(pose_graph)

In [ ]:
import gtsam
import numpy as np

# Create a factor graph container and add factors to it
factor_graph = gtsam.NonlinearFactorGraph()

# Add prior
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(0.001 * np.ones(6))
factor_graph.add(gtsam.PriorFactorPose3(0, pose_graph.get_node_pose(0), PRIOR_NOISE))

# Add odometry factors
for e in graph.edges:
    if e["type"] == "odometry":
        ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Information(e["info"])
        factor_graph.add(gtsam.BetweenFactorPose3(e["parent_id"], e["child_id"], e["pose"], ODOMETRY_NOISE))

# Add initial values
initial_estimate = gtsam.Values()
for i, node in enumerate(graph.nodes):
    initial_estimate.insert(i, graph.get_node_pose(i))

#### Visualize factor graph

In [ ]:
factor_graph.saveGraph("test.dot", initial_estimate)
from graphviz import Source
s = Source.from_file("test.dot")
s.view()

#### Optimize

In [ ]:
# Setup optimizer
parameters = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(factor_graph, initial_estimate, parameters)

import open3d as o3d
import time
import copy

visualizer = o3d.visualization.Visualizer()
visualizer.create_window()

graph_vis = copy.deepcopy(pose_graph)
# Manually run optimizer
for i in range(10):
    print(f"iter: {i}")
    visualizer.clear_geometries()
    geometry = vis.to_geometries(graph_vis)
    for g in geometry:
        visualizer.add_geometry(g)
    
    optimizer.iterate()
    sol = optimizer.values()
    for n,_ in enumerate(graph_vis.nodes):
        graph_vis.set_node_pose(n, sol.atPose3(n))
    
    visualizer.poll_events()
    visualizer.update_renderer()
    time.sleep(1)

In [ ]:
# Compute marginals to show covariances
marginals = gtsam.Marginals(factor_graph, optimizer.values())